In [3]:
#!pip3 install opencv-python

from gym.wrappers import FrameStack, GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from matplotlib import pyplot as plt

import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

env = gym_super_mario_bros.make('SuperMarioBros-v0')
env= JoypadSpace(env, SIMPLE_MOVEMENT);

env = GrayScaleObservation(env, keep_dim=True);
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

state = env.reset();
state.shape


(1, 240, 256, 4)

In [4]:
import os

from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback

class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path=save_path
    
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok =True)
    
    def _on_step(self):
        if self.n_calls % self.check_freq ==0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
        return True;


In [5]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

callback = TrainAndLoggingCallback(check_freq = 100000, save_path= CHECKPOINT_DIR)


In [8]:
#!pip install tensorboard
model = PPO('CnnPolicy', env, verbose =1, tensorboard_log=LOG_DIR, learning_rate=0.00001, n_steps=512)

model.learn(total_timesteps=1000000, callback=callback)
print('done learning')

Using cpu device
Wrapping the env in a VecTransposeImage.
Logging to ./logs/PPO_1


/opt/anaconda3/lib/python3.8/site-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


----------------------------
| time/              |     |
|    fps             | 81  |
|    iterations      | 1   |
|    time_elapsed    | 6   |
|    total_timesteps | 512 |
----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 11           |
|    iterations           | 2            |
|    time_elapsed         | 87           |
|    total_timesteps      | 1024         |
| train/                  |              |
|    approx_kl            | 0.0026590752 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.94        |
|    explained_variance   | -0.00118     |
|    learning_rate        | 1e-05        |
|    loss                 | 92           |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00281     |
|    value_loss           | 214          |
------------------------------------------
-----------------------

-----------------------------------------
| time/                   |             |
|    fps                  | 6           |
|    iterations           | 13          |
|    time_elapsed         | 967         |
|    total_timesteps      | 6656        |
| train/                  |             |
|    approx_kl            | 0.002381351 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.88       |
|    explained_variance   | -0.000566   |
|    learning_rate        | 1e-05       |
|    loss                 | 0.0396      |
|    n_updates            | 120         |
|    policy_gradient_loss | -0.000895   |
|    value_loss           | 0.156       |
-----------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 6            |
|    iterations           | 14           |
|    time_elapsed         | 1048         |
|    total_timesteps      | 7

/opt/anaconda3/lib/python3.8/site-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


------------------------------------------
| time/                   |              |
|    fps                  | 6            |
|    iterations           | 20           |
|    time_elapsed         | 1497         |
|    total_timesteps      | 10240        |
| train/                  |              |
|    approx_kl            | 0.0039871796 |
|    clip_fraction        | 0.0166       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.81        |
|    explained_variance   | 0.748        |
|    learning_rate        | 1e-05        |
|    loss                 | 0.519        |
|    n_updates            | 190          |
|    policy_gradient_loss | -0.00347     |
|    value_loss           | 2.13         |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 6            |
|    iterations           | 21           |
|    time_elapsed         | 1568         |
|    total_

KeyboardInterrupt: 

In [ ]:
# loaded_model = PPO.load('./train/best_model_10000')
# state=env.reset();
# while True:
#     action,_state = model.predict(state)
#     state, reward, done, info = env.step(action)
#     env.render();